Joey Demple<br>
Prof Hulden<br>
LING/CSCI 5832<br>
6 April 2018<br>
Homework 5: PCFG Parsing with the CKY Algorithm

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 200)  # display entire set of rules and proababilities in each cell

In [2]:
grammar = {
    ('S','NP','VP'):0.9,
    ('S','VP'):0.1,
    ('VP','V','NP'):0.5,
    ('VP','V'):0.1,
    ('VP','V','@VP_V'):0.3,
    ('VP','V','PP'):0.1,
    ('@VP_V','NP','PP'):1.0,
    ('NP','NP','NP'):0.1,
    ('NP','NP','PP'):0.2,
    ('NP','N'):0.7,
    ('PP','P','NP'):1.0,
    ('N','people'):0.5,
    ('N','fish'):0.2,
    ('N','tanks'):0.2,
    ('N','rods'):0.1,
    ('V','people'):0.1,
    ('V','fish'):0.6,
    ('V','tanks'):0.3,
    ('P','with'):1.0
}

In [12]:
def CKY(sentence, grammar):
    #table = init_table(sentence)
    table = pd.DataFrame(index=range(len(sentence)), columns=range(len(sentence)))
    
    
    #### First Loop (Outer Loop) ####
    i = 0
    for word in sentence:
        
        # terminal loop
        terminal_probs = []
        for rule in grammar:
            if word in rule:
                terminal_probs.append((rule, grammar[rule]))
        
        # add unary probabilities
        unary_probs = []
        for tp in terminal_probs:
            for rule in grammar:
                if tp[0][0] == rule[-1]:
                    # update probability and append to unary list
                    updated_prob = grammar[rule] * grammar[tp[0]]
                    unary_probs.append((rule, updated_prob))
        
        # check to see if unaries have any unaries above
        for up in unary_probs:
            for rule in grammar:
                if up[0][0] == rule[-1]:
                    if len(rule) == 2:
                        # update probability and append to unary list
                        updated_prob = grammar[rule] * up[1]
                        unary_probs.append((rule, updated_prob))

        probs = terminal_probs + unary_probs
        table.at[i,i] = probs
        i += 1
    
    
    #### Main CKY Loop ####
    span_loop = 1  # Which loop we are on; span increases by one along with span_loop
    for span in range(2, len(sentence)+1):
        #print('SPAN', span)
        #### Span loop #### 
        for span_start in range(len(sentence)-span_loop):
            #### Cell Loop ####
            #print('span_start', span_start)
            span_end = span_start + span
            #print('span_end', span_end)
            # Write location AKA cell location in table we are currently working on
            write_coord = (span_start, span_start + span - 1)
            #print('write_coord', write_coord)
            
            test = []
            binary_matches = []
            unary_matches = []
            
            for i in range(1, span):
                #### Split Possibilites Loop ####
                #print('i\t', i)
                left_cell_loc = (span_start, span_start +i -1)
                right_cell_loc = (span_end- (span_end - i) + span_start, span_end -1)
                #print('LOC', left_cell_loc, right_cell_loc)
                test.append((left_cell_loc, right_cell_loc))
                
                left_probs = table.at[left_cell_loc]
                right_probs = table.at[right_cell_loc]
                
                #print('left_probs', left_probs)
                #print('right_probs', right_probs)
                
                #### Binary search ####
                ## Loop Through all binary rules
                ## if the left tail of rule matches a head in the left_probs  
                ## and the right tail of rule matches a head in the right_probs
                ## then update probability and add to the list of our rule,probability
                ## pairs to be written in this cell
                #print('BINARY RULES')
                for rule in grammar:
                    if len(rule) == 3:  # if rule is binary (A -> B C)
                        for left_pair in left_probs:
                            if rule[1] == left_pair[0][0]:  # if B is a head in left pairs
                                for right_pair in right_probs:
                                    if rule[2] == right_pair[0][0]: # if C is a head in right pairs
                                        # Update probability
                                        updated_prob = grammar[rule] * left_pair[1] * right_pair[1]
                                        binary_matches.append((rule, updated_prob))
                        
                        
                
                #### Unary Search ####
                ## Loop through all binary rule,prob pairs we just added to binary_matches
                ## for each pair, if the head of the rule in our list matches the tail of
                ## a unary rule in the grammar, then we update the proability and add 
                ## this rule,prob pair to our list of unary_matches
                #print('UNARY RULES')
                for binary_pair in binary_matches:
                    #print(binary_pair)
                    for rule in grammar:
                        if len(rule) == 2:  # if rule is unary
                            if binary_pair[0][0] == rule[-1]:
                                updated_prob = grammar[rule] * binary_pair[1]
                                unary_matches.append((rule, updated_prob))
                for unary_pair in unary_matches:
                    #print(unary_pair)
                    for rule in grammar:
                        if len(rule) == 2:  # if rule is unary
                            if unary_pair[0][0] == rule[-1]:
                                updated_prob = grammar[rule] * unary_pair[1]
                                unary_matches.append((rule, updated_prob))
            #print('unary_matches',unary_matches)
            #print('binary_matches', binary_matches)
            #### Dedupe Where Rule Heads Are Repeated ####
            #total_matches = binary_matches + unary_matches
            total_matches = binary_matches + list(unary_matches)
            #print('TOTAL MATCHES', total_matches)
            head_set = [match[0][0] for match in total_matches]
            #print(head_set)
            for head in set(head_set):
                if head_set.count(head) > 1:  # if duplicate
                    # keep highest and drop the rest
                    rule_list = [match[0] for match in total_matches if match[0][0] == head]
                    prob_list = [match[1] for match in total_matches if match[0][0] == head]                    
                    prob, ix, = max([(v,i) for i,v in enumerate(prob_list)])
                    #print('prob, ix', prob, ix)
                    prob_to_keep = (rule_list[ix], prob_list[ix])
                    #print('prob_to_keep', prob_to_keep)

                    # filter to new prob list with only the highest one using this head
                    total_matches = [match for match in total_matches if match[0][0] != head]
                    total_matches.append(prob_to_keep)
                   
            
            # Write to Cell
            #print('TOTAL MATCHES DEDUPED', total_matches)
            table.at[write_coord] = total_matches
        
        # move to next loop and increase span_loop (span var will increase automatically so don't do it here)
        span_loop += 1
        #print('\n')
    
    # return parse probability
    for pair in total_matches:
        if pair[0][0] == 'S':
            return pair[1]
    else:
        # No possible parse
        return 0.0

## Demonstration

In [14]:
print(CKY(['fish','people','fish','tanks'], grammar))
print(CKY(['people','with','fish','rods','fish','people'], grammar))
print(CKY(['fish','with','fish','fish'], grammar))
print(CKY(['fish','with','tanks','people','fish'], grammar))
print(CKY(['fish','people','with','tanks','fish','people','with','tanks'], grammar))
print(CKY(['fish','fish','fish','fish','fish'], grammar))
print(CKY(['rods','rods','rods','rods'], grammar))

0.00018521999999999994
6.482699999999997e-06
0.00021167999999999992
2.4695999999999992e-05
1.0890935999999993e-06
1.0372319999999996e-06
0.0
